# 신호등 데이터 수집을 위한 ipywidgets 사용하기
카메라를 초기화하고 화면을 표시합니다.

신경망은 224 x 224 픽셀 이미지를 입력으로 사용하기 때문에 파일 크기를 최소화하고 데이터 세트를 최소화하기 위해 카메라를 이 크기로 설정합니다. (이 작업을 위해 이 픽셀을 테스트했습니다.) 데이터를 수집할 때는 이미지 크기를 크게 사용하고 처리할 때는 필요한 크기로 줄이는 것이 가장 좋습니다.

In [2]:
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
from IPython.display import display

import traitlets

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

데이터를 저장하기 위해 몇 가지 디렉토리를 만들어야 합니다. 각 장면의 이미지를 정렬하기 위해 자유롭고 차단된 두 개의 하위 폴더가 있는 데이터 세트라는 폴더를 만들 것입니다.

In [3]:
import os

red_dir = 'dataset/red'
green_dir = 'dataset/green'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(red_dir)
    os.makedirs(green_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

왼쪽에 있는 Jupyter 파일 브라우저를 새로 고치면 이제 이러한 디렉토리가 표시됩니다. 다음으로 각 클래스 태그에 대한 스냅샷을 저장하는 데 사용되는 몇 가지 버튼을 생성하여 표시합니다. 지금까지 수집한 각 범주의 이미지 수를 표시하는 텍스트 상자도 추가합니다. 이는 "블록" 이미지 수만큼의 "사용 가능한" 이미지를 수집하고 싶으므로 유용합니다. 또한 총 몇 개의 이미지를 수집했는지 파악하는 데 도움이 됩니다.

In [60]:
button_layout = widgets.Layout(width='128px', height='3px')

red_button = widgets.Button(description='add red', button_style='danger', layout=button_layout)
red_count = widgets.IntText(layout=button_layout, value=len(os.listdir(red_dir)))

green_button = widgets.Button(description='add green', button_style='success', layout=button_layout)
green_count = widgets.IntText(layout=button_layout, value=len(os.listdir(green_dir)))

display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([green_count, green_button]))

현재 이 버튼들은 아무 것도 할 수 없습니다. 'n_click' 버튼 카테고리별로 이미지를 저장하는 기능을 첨부해야 합니다. 이미 압축된 JPEG 형식이기 때문에 '이미지' 위젯의 값을 저장합니다!

다른 컴퓨터에서도 파일 이름이 반복되지 않도록 하기 위해 python의 "uuid" 패키지를 사용하여 고유 식별자를 생성합니다. 이 고유 식별자는 현재 시간 및 컴퓨터 주소와 같은 정보에서 생성됩니다.

In [59]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)
        
def save_red():
    global red_dir, red_count
    save_snapshot(red_dir)
    red_count.value = len(os.listdir(red_dir))
    
def save_green():
    global green_dir, green_count
    save_snapshot(green_dir)
    green_count.value = len(os.listdir(green_dir))
    
# red_button.on_click(lambda x: save_red())
# green_button.on_click(lambda x: save_green())

def on_click_save_red(x):
    save_red()

def on_click_save_green(x):
    save_green()
    
red_button.on_click(on_click_save_red)
green_button.on_click(on_click_save_green)

이제 위의 버튼으로 이미 이미지를 "자유" 디렉토리와 "차단" 디렉토리에 저장할 수 있습니다. 이 파일들은 주피터 왼쪽 디렉토리 파일 브라우저를 사용하여 볼 수 있습니다!

이제 몇 가지 데이터를 계속 수집합니다:
1. 로봇을 차단된 장면에 놓고 "차단 추가"를 누릅니다.
2. 로봇을 자유 장면에 놓고 "자유 추가"를 누릅니다.
3. 1번, 2번 반복합니다
(알림: 셀을 마우스 오른쪽 단추로 클릭하고 출력에 대한 새 보기 만들기를 클릭하여 위젯을 새 창으로 이동할 수 있습니다.)

데이터에 태그를 붙이는 몇 가지 팁은 다음과 같습니다:
여러 방향을 시도해 봅니다
2. 다른 조명을 사용해 봅니다
3. 벽, 바위 선반, 객체 등 다양한 객체/충돌 유형 시도
4. 다른 질감의 바닥/물건을 사용해 보십시오. 패턴, 매끄러운 모양, 유리 모양 등이 있습니다.

많은 데이터뿐만 아니라 다양한 데이터(위의 팁에서 설명한 바와 같이)를 얻는 것이 중요하며, 클래스당 최소 100개의 이미지가 필요할 수도 있습니다(이것은 과학이 아니라 유용한 트릭입니다).

아래 셀 코드를 실행하면 이미지와 버튼이 표시되고 데이터 수집을 시작할 수 있습니다:

충분한 데이터를 수집할 때는 교육을 위해 이 데이터를 GPU 플랫폼에 복사해야 합니다.

먼저 terminal(명령줄 모드 또는 terminal) 명령을 호출하여 데이터를 *.zip 파일로 압축할 수 있습니다.

! 셸 명령을 사용하여 실행하고, -r은 모든 하위 폴더 파일을 포함하고, -q는 zip 명령이 어떤 정보도 출력하지 않음을 의미합니다.

In [ ]:
!zip -r -q dataset.zip dataset

When you run the code in the above cell, you will see a file named dataset.zip in the Jupyter Lab file browser. You can right click on the file to download it.